In [1]:
import sys
sys.path.append("../")
import os
import vqe_experiment as ve
import numpy as np

In [2]:
# define Hamiltonian: list of coefficients, list of Paulis, initial bitstring

# here: molecular Hamiltonian, parity mapping, two-qubit reduction
atom_string = "Li 0 0 0; H 0 0 2.0"              # PySCF atom string
num_orbitals = 3                                 # number of spatial orbitals
coeffs, paulis, HF_bitstring = ve.molecule(atom_string, new_num_orbitals=num_orbitals)

In [3]:
coeffs, paulis, HF_bitstring

(array([-5.59396305e+00,  1.61387904e+00, -1.72701988e-02,  1.72701988e-02,
        -3.86183811e-02, -3.86183811e-02,  3.90387222e-01,  2.52944297e-01,
        -1.13923657e-02,  1.13923657e-02,  1.61387904e+00,  4.14787999e-01,
        -2.50295510e-02,  2.50295510e-02, -3.52792774e-02, -3.52792774e-02,
        -1.72701988e-02, -2.50295510e-02,  1.72701988e-02,  2.50295510e-02,
         2.63398182e-03, -2.63398182e-03, -2.63398182e-03,  2.63398182e-03,
         2.65122860e-03, -2.65122860e-03,  2.65122860e-03, -2.65122860e-03,
        -3.86183811e-02, -3.52792774e-02, -3.86183811e-02, -3.52792774e-02,
         2.65122860e-03,  2.65122860e-03, -2.65122860e-03, -2.65122860e-03,
         5.49722197e-03,  5.49722197e-03,  5.49722197e-03,  5.49722197e-03,
         3.90387222e-01,  8.14827873e-02,  8.55526893e-04, -8.55526893e-04,
        -3.05064421e-03, -3.05064421e-03, -1.13923657e-02,  5.87484994e-03,
         1.13923657e-02, -5.87484994e-03, -6.66607492e-04,  6.66607492e-04,
         6.6

In [4]:
# number of qubits
N = len(paulis[0])

In [5]:
# configure underlying VQE structure, passed as keyword argument dictionary

# a function which returns a parameterized Qiskit circuit (ansatz) and the number of parameters
ansatz_func = ve.efficientsu2_full
ansatz_reps = 1
# if True, the initial HF_bitstring will be appended after ansatz (layer of X gates where there are 1s)
# if False, HF_bitstring will be initialized before ansatz
init_last = False

vqe_kwargs = {
    "ansatz_func": ansatz_func,
    "ansatz_reps": ansatz_reps,
    "init_last": init_last,
    "HF_bitstring": HF_bitstring
}

In [6]:
ansatz, num_params = ansatz_func(N, ansatz_reps)
print(num_params)

16


In [7]:
ansatz.draw(fold=-1)

┌──────────┐┌──────────┐ ░                                ░  ┌──────────┐┌───────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├─░───■────■────■──────────────────░──┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├
     ├──────────┤├──────────┤ ░ ┌─┴─┐  │    │                  ░  ├──────────┤├───────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├──┼────┼────■────■────────░──┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├
     ├──────────┤├──────────┤ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░ ┌┴──────────┤├───────────┤
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░─┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├
     ├──────────┤├──────────┤ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ├───────────┤├───────────┤
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─░───────────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├
     └──────────┘└──────────┘ ░           └───┘     └───┘└───┘ ░ └───────────┘└───────────┘

CAFQA parameters are represented by a list $[x_0, x_1, x_2, ...]$ where $x_i$ parametrizes the $i$-th gate in the ansatz and can only take the discrete values $\{0,1,2,3\}$. Typically the parameterized gates are rotation gates, $R_y$ or $R_z$, so that $x_i$ corresponds to a rotation angle $x_i \cdot \pi/2$.

In [8]:
# configure algorithm

budget = 100                                                 # number of iterations
guess = list(map(int, np.random.randint(0, 4, num_params)))  # initial param guess
guess

[3, 1, 1, 1, 1, 3, 3, 1, 1, 0, 2, 1, 1, 1, 1, 3]

In [9]:
# get VQE circuit guess parameters (* pi/2)
qc = ve.vqe_circuit(
    N, 
    np.array(guess)*np.pi/2, 
    paulis, 
    ansatz_func=ansatz_func, 
    ansatz_reps=ansatz_reps, 
    init_last=init_last,
    HF_bitstring=HF_bitstring
    )

In [10]:
qc.draw(fold=-1)

┌───┐   ┌──────────┐┌─────────┐  ░                                ░ ┌─────────┐┌─────────┐ 
q1_0: ───┤ X ├───┤ Ry(3π/2) ├┤ Rz(π/2) ├──░───■────■────■──────────────────░─┤ Ry(π/2) ├┤ Rz(π/2) ├─
      ┌──┴───┴──┐├──────────┤└─────────┘  ░ ┌─┴─┐  │    │                  ░ └┬───────┬┘├─────────┤ 
q1_1: ┤ Ry(π/2) ├┤ Rz(3π/2) ├─────────────░─┤ X ├──┼────┼────■────■────────░──┤ Ry(0) ├─┤ Rz(π/2) ├─
      └──┬───┬──┘├─────────┬┘┌──────────┐ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░  ├───────┤ ├─────────┤ 
q1_2: ───┤ X ├───┤ Ry(π/2) ├─┤ Rz(3π/2) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░──┤ Ry(π) ├─┤ Rz(π/2) ├─
      ┌──┴───┴──┐├─────────┤ └──────────┘ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ┌┴───────┴┐├─────────┴┐
q1_3: ┤ Ry(π/2) ├┤ Rz(π/2) ├──────────────░───────────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(π/2) ├┤ Rz(3π/2) ├
      └─────────┘└─────────┘              ░           └───┘     └───┘└───┘ ░ └─────────┘└──────────┘
c0: 4/══════════════════════════════════════════════════════════════════════════════════════════════

In [11]:
# configure output

save_dir = "cafqa_example_out/"
os.system(f"mkdir -p {save_dir}")
# the output files are relative to save_dir
result_file = "result.txt"
loss_file = "cafqa_loss.txt"
params_file = "cafqa_params.txt"

In [12]:
# run CAFQA

# hypermapper changes the standard output to write to log files, so it needs to be
# changed back after CAFQA (otherwise printing breaks)
stdout = sys.stdout
cafqa_energy, cafqa_params = ve.run_cafqa(
    n_qubits=N,
    coeffs=coeffs,
    paulis=paulis,
    param_guess=list(guess),
    budget=budget,
    save_dir=save_dir,
    loss_file=loss_file,
    params_file=params_file,
    vqe_kwargs=vqe_kwargs
)
sys.stdout = stdout

Design of experiment phase, number of new doe samples = 100 .......
Loss computed by CAFQA VQE is -4.76339385879937, in 0.05460954099999782 s.
Loss computed by CAFQA VQE is -4.7966851504455885, in 0.04131854199999907 s.
Loss computed by CAFQA VQE is -4.991349030109287, in 0.04611720800000185 s.
Loss computed by CAFQA VQE is -4.843937726263331, in 0.03785404199999931 s.
Loss computed by CAFQA VQE is -6.43043486417881, in 0.03316091700000001 s.
Loss computed by CAFQA VQE is -5.013861735218722, in 0.03588970799999913 s.
Loss computed by CAFQA VQE is -3.0307483515954625, in 0.029166082999999787 s.
Loss computed by CAFQA VQE is -4.79690901007773, in 0.04852725000000291 s.
Loss computed by CAFQA VQE is -4.854826580768879, in 0.03959516600000157 s.
Loss computed by CAFQA VQE is -4.712639406672185, in 0.04047479200000126 s.
Loss computed by CAFQA VQE is -4.967417701721457, in 0.03804745900000128 s.
Loss computed by CAFQA VQE is -3.18355600728451, in 0.036244583000001995 s.
Loss computed by CAF

AttributeError: Can't pickle local object 'Space.__init__.<locals>.<lambda>'